In [1]:
import pandas as pd
from collections import Counter

ed_df = pd.read_csv("data/encoder_decoder.csv")
ed_df["params"] = "7.4M"
ed_df["arch_name"] = "encoder-decoder"
print(len(set(ed_df["Name"])))
Counter(ed_df["Name"]).most_common(1)

52


[('albanian', 1)]

In [2]:
d48_df = pd.read_csv("data/decoder_only_4layers_8heads.csv")
d48_df["params"] = "3.2M"
d48_df["encoder_layers"] = 0
d48_df["arch_name"] = "decoder-only-4layer-8head"
print(len(set(d48_df["Name"])))
Counter(d48_df["Name"]).most_common(1)

52


[('georgian', 1)]

In [3]:
d84_df = pd.read_csv("data/decoder_only_8layers_4heads.csv")
d84_df["params"] = "6.3M"
d84_df["encoder_layers"] = 0
d84_df["arch_name"] = "decoder-only-8layer-4head"
print(len(set(d84_df["Name"])))
Counter(d84_df["Name"]).most_common(1)

52


[('welsh', 1)]

In [4]:
d44_df = pd.read_csv("data/decoder_only_4layers_4heads.csv")
d44_df["params"] = "3.2M"
d44_df["encoder_layers"] = 0
d44_df["arch_name"] = "decoder-only-4layer-4head"
print(len(set(d44_df["Name"])))
Counter(d44_df["Name"]).most_common(1)

52


[('albanian', 1)]

In [5]:
d88_df = pd.read_csv("data/decoder_only_8layers_8heads.csv")
d88_df["params"] = "6.4M"
d88_df["encoder_layers"] = 0
d88_df["arch_name"] = "decoder-only-8layer-8head"
print(len(set(d88_df["Name"])))
Counter(d88_df["Name"]).most_common(1)

52


[('albanian', 1)]

In [6]:
all_df = pd.concat([ed_df, d48_df, d84_df, d44_df, d88_df])

all_df.groupby(["arch_name", "params"]).mean()[[
    "val_accuracy.max",
    "source_attention_heads",
    "decoder_layers",
    "encoder_layers",
]].sort_values("val_accuracy.max")

,,val_accuracy.max,source_attention_heads,decoder_layers,encoder_layers
arch_name,params,,,,
decoder-only-4layer-4head,3.2M,0.933923,4.0,4.0,0.0
decoder-only-4layer-8head,3.2M,0.942096,8.0,4.0,0.0
decoder-only-8layer-4head,6.3M,0.951462,4.0,8.0,0.0
decoder-only-8layer-8head,6.4M,0.951558,8.0,8.0,0.0
encoder-decoder,7.4M,0.961250,4.0,4.0,4.0


In [7]:
max_arches = all_df.sort_values("val_accuracy.max", ascending=False).drop_duplicates(["Name"]).sort_values("Name")[
    ["Name", "arch_name", "val_accuracy.max"]
]
maxes = max_arches[["Name", "val_accuracy.max"]]
bests = []
for name in sorted(set(maxes["Name"])):
    best = maxes.loc[maxes["Name"] == name]["val_accuracy.max"].iloc[0]
    name_df = all_df.loc[all_df["Name"] == name]
    for idx, row in name_df.iterrows():
        row_acc = row["val_accuracy.max"]
        if row_acc == best:
            bests.append(row["arch_name"])
            
print(Counter(bests))

Counter({'encoder-decoder': 38, 'decoder-only-8layer-4head': 14, 'decoder-only-8layer-8head': 9, 'decoder-only-4layer-8head': 7, 'decoder-only-4layer-4head': 6})


In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(all_df.groupby(["Name", "arch_name"])["val_accuracy.max"].max()))

                                             val_accuracy.max
Name              arch_name                                  
albanian          decoder-only-4layer-4head             0.952
                  decoder-only-4layer-8head             0.944
                  decoder-only-8layer-4head             0.964
                  decoder-only-8layer-8head             0.972
                  encoder-decoder                       0.997
arabic            decoder-only-4layer-4head             0.966
                  decoder-only-4layer-8head             0.970
                  decoder-only-8layer-4head             0.969
                  decoder-only-8layer-8head             0.971
                  encoder-decoder                       0.972
armenian          decoder-only-4layer-4head             0.969
                  decoder-only-4layer-8head             0.969
                  decoder-only-8layer-4head             0.970
                  decoder-only-8layer-8head             0.974
        

In [9]:
ties = {}

for idx, row in all_df.iterrows():
    ties.setdefault(row.Name, {})
    ties[row.Name].setdefault(row["val_accuracy.max"], []).append(row["arch_name"])
    
for name, d in ties.items():
    for k, v in d.items():
        if len(v) > 1 and k == max(d.keys()):
            print(name, v, k)

basque ['encoder-decoder', 'decoder-only-8layer-4head', 'decoder-only-8layer-8head'] 0.9900000095367432
bengali ['encoder-decoder', 'decoder-only-4layer-8head', 'decoder-only-8layer-4head', 'decoder-only-4layer-4head', 'decoder-only-8layer-8head'] 0.9900000095367432
danish ['encoder-decoder', 'decoder-only-8layer-8head'] 0.9629999995231628
finnish ['decoder-only-8layer-4head', 'decoder-only-4layer-4head'] 0.9639999866485596
haida ['encoder-decoder', 'decoder-only-4layer-8head', 'decoder-only-8layer-4head', 'decoder-only-8layer-8head'] 0.9900000095367432
hindi ['encoder-decoder', 'decoder-only-4layer-8head', 'decoder-only-4layer-4head'] 1.0
kurmanji ['encoder-decoder', 'decoder-only-4layer-4head'] 0.9350000023841858
latvian ['decoder-only-8layer-4head', 'decoder-only-4layer-4head'] 0.9829999804496764
persian ['encoder-decoder', 'decoder-only-4layer-8head'] 1.0
portuguese ['encoder-decoder', 'decoder-only-8layer-4head'] 0.9929999709129332
quechua ['encoder-decoder', 'decoder-only-4layer-

# 2023

In [10]:
import pandas as pd
from collections import Counter

ed_23_df = pd.read_csv("data/2023-encoder-decoder.csv")
ed_23_df["params"] = "7.4M"
ed_23_df["arch_name"] = "encoder-decoder"
print(len(set(ed_23_df["Name"])))
Counter(ed_23_df["Name"]).most_common(1)

27


[('sqi.trn', 1)]

In [11]:
d84_23_df = pd.read_csv("data/2023-decoder-only-8layers-4heads.csv")
d84_23_df["params"] = "6.3M"
d84_23_df["encoder_layers"] = 0
d84_23_df["arch_name"] = "decoder-only-8layer-4head"
print(len(set(d84_23_df["Name"])))
Counter(d84_23_df["Name"]).most_common(7)

26


[('dan.trn', 1),
 ('bel.trn', 1),
 ('afb.trn', 1),
 ('arz.trn', 1),
 ('tur.trn', 1),
 ('sqi.trn', 1),
 ('swa.trn', 1)]

In [12]:
all_2023_df = pd.concat([ed_23_df, d84_23_df])

all_2023_df.groupby(["arch_name", "params"]).mean()[[
    "val_accuracy.max",
    "source_attention_heads",
    "decoder_layers",
    "encoder_layers",
]].sort_values("val_accuracy.max")

,,val_accuracy.max,source_attention_heads,decoder_layers,encoder_layers
arch_name,params,,,,
decoder-only-8layer-4head,6.3M,0.323308,4.0,8.0,0.0
encoder-decoder,7.4M,0.846259,4.0,4.0,4.0


In [16]:
d84_df.loc[d84_df["Name"] == "german"]

,Name,epoch,val_accuracy.max,State,Created,Runtime,Sweep,arch,beam_width,beta1,...,source_vocab_size,start_idx,target_vocab_size,teacher_forcing,lr-Adam,train_loss,trainer/global_step,val_loss,params,arch_name
35,german,799,0.912,finished,2023-12-15T02:33:20.000Z,5585,NaN,decoder_only_transformer,1,0.9,...,91,2,71,True,0.000447,0.756447,19999,0.911133,6.3M,decoder-only-8layer-4head
